# PRÁCTICA 1 MEDICIÓN E INSTRUMENTACIÓN
# Análisis de error aleatorio y ruido 

<font color='blue' size="6">ACTIVIDAD 1. Generación de una señal aleatoria</font> 

## Paso 1: Importar bibliotecas para números, matemáticas y gráficas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from scipy import stats 
from scipy.stats import norm

## Paso 2: Generar una secuencia aleatoria con $N$ muestras y distribución normal

In [ ]:
N = 1024 #número de muestras
noise = np.random.normal(0,1,1024) #secuencia aleatoria con distribución normal
print(len(noise)) #imprime la longitud de la secuencia (para verificar)
print(noise) #imprime los datos generados

## Paso 3: Graficar la secuencia noise en función del número de muestras $N$

In [ ]:
plt.plot(noise)
plt.xlabel('Muestras')
plt.ylabel('Amplitud [u.a]')

In [ ]:
mu=np.mean(noise) # media

sigma=np.std(noise) #desviación estándar

std_err = sigma / N # error estándar

print( 'media: ', mu)
print( 'desviacion estandar: ', sigma)
print( 'error estandar: ', std_err)

In [ ]:
bins=10 # número de bins del histograma
n,bin_positions,p = plt.hist(noise ,color = 'blue', edgecolor = 'black', bins = int(180/5)) # grafico el histograma

bin_size=bin_positions[1]-bin_positions[0] # calculo el ancho de los bins del histograma

x_gaussiana=np.linspace(mu-5*sigma,mu+5*sigma,num=100) # armo una lista de puntos donde quiero graficar la distribución de ajuste

gaussiana=norm.pdf(x_gaussiana, mu, sigma)*N*bin_size # calculo la gaussiana que corresponde al histograma

plt.plot(x_gaussiana,gaussiana,'r--', linewidth=3, label='ajuste 1') #grafico la gaussiana

plt.show()

<font color='blue' size="6">ACTIVIDAD 2. Un ejemplo real: RUIDO</font> 

## La tarjeta de audio de una computadora es un dispositivo con convertidores analógico a digital (ADC) - micrófono -, y digital a analógico (DAC) - bocinas -. 
## Mediante el ADC podemos adquirir datos de señales continuas, veamos que pasa ...

## Paso 1: Instalar el paquete asociado a la tarjeta de audio (pyaudio)

In [ ]:
import sys
!pip install pyaudio

In [ ]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
from tkinter import TclError

# para observar la señal en una ventana separada
%matplotlib tk

# constants
CHUNK = 1024 * 2             # 2048 muestras por cuadro
FORMAT = pyaudio.paInt16     # formato de audio Entero 16 bits
CHANNELS = 1                 # Un solo canal 
RATE = 44100                 # Frecuencia de muestreo

In [ ]:
# crea una figura y sus ejes
fig, ax = plt.subplots(1, figsize=(5, 3))

In [ ]:
# instancia de una clase de pyaudio
p = pyaudio.PyAudio()

# objeto para adquisición de datos
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

In [ ]:
# variable para graficar
x = np.arange(0, 2 * CHUNK, 2)

# inicia la grafica con una señal aleatoria
line, = ax.plot(x, np.random.rand(CHUNK), '-', lw=2)

# formato de los ejes de la grafica
ax.set_title('Señal de audio')
ax.set_xlabel('Muestras')
ax.set_ylabel('Amplitud')
ax.set_ylim(0, 255)
ax.set_xlim(0, 2 * CHUNK)
plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])


In [ ]:
# muestra la gráfica
plt.show(block=False)

print('adquisicion iniciada')

# medición del tiempo
frame_count = 0
start_time = time.time()

#realiza 100 mediciones de paquetes de 2048 datos
i = 1
while i < 101:
   
    # datos en formato binario
    data = stream.read(CHUNK)  
    
    # convierte a enteros, y crea un arreglo
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # añade offset de 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    line.set_ydata(data_np)
    i += 1
    # actualiza la figura continuamente
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break


In [ ]:
print(data_int)

In [ ]:
mu=np.mean(data_np) # media

sigma=np.std(data_np) #desviación estándar

std_err = sigma / N # error estándar

print( 'media: ', mu)
print( 'desviacion estandar: ', sigma)
print( 'error estandar: ', std_err)

In [ ]:
bins=10 # número de bins del histograma
%matplotlib inline
n,bin_positions,p = plt.hist(data_np ,color = 'blue', edgecolor = 'black', bins = int(180/5)) # grafico el histograma
bin_size=bin_positions[1]-bin_positions[0] # calculo el ancho de los bins del histograma

x_gaussiana=np.linspace(mu-5*sigma,mu+5*sigma,num=100) # armo una lista de puntos donde quiero graficar la distribución de ajuste

gaussiana=norm.pdf(x_gaussiana, mu, sigma)*N*bin_size # calculo la gaussiana que corresponde al histograma

plt.plot(x_gaussiana,gaussiana,'r--', linewidth=3, label='ajuste 1') #grafico la gaussiana

plt.show()